
https://www.kaggle.com/code/mistag/keras-model-tuning-with-optuna/notebook

https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im

In [9]:
import os; os.chdir("/home/mert/Desktop/thesis")
from scripts.window import WindowGenerator
os.chdir("/home/mert/Desktop/thesis/notebooks")

In [10]:
import warnings; warnings.filterwarnings("ignore")

In [11]:
# !pip install optuna
import optuna

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.backend import clear_session
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import models, layers

In [12]:
GENERATE_SPEED_ANGLE = True
add_lagged = True
INPUT_WIDTH = 144

train_ratio = 0.8
valid_ratio = 0.1

df = pd.read_parquet("../data/processed/outlier_removed.parquet")

if add_lagged:
    df["production_48_lagged"] = df.groupby("rt_plant_id").production.shift(48)

SELECTED_PLANT_ID = df.groupby("rt_plant_id").production.sum().sort_values(ascending=False).index[0]
print(SELECTED_PLANT_ID)
weather_cols = [col for col in df.columns if col.startswith(("UGRD", "VGRD"))]
if add_lagged:
    df = df.set_index("forecast_dt")[["rt_plant_id", "production", "production_48_lagged", *weather_cols]]
    df = df.dropna()
else:    
    df = df.set_index("forecast_dt")[["rt_plant_id", "production", *weather_cols]]
df = df[df["rt_plant_id"] == SELECTED_PLANT_ID].drop("rt_plant_id", axis=1)
if GENERATE_SPEED_ANGLE:
    for box in ["SW", "NW", "NE", "SE"]:
        df[f"speed_{box}"] = np.sqrt(np.square(df[f"UGRD_80.m.above.ground.{box}"]) + np.square(df[f"VGRD_80.m.above.ground.{box}"]))
        df[f"angle_{box}"] = np.arctan(df[f"UGRD_80.m.above.ground.{box}"] / df[f"VGRD_80.m.above.ground.{box}"])
        
time_indices = sorted(df.index.unique())

train_indices = time_indices[:int(len(time_indices) * train_ratio)]
valid_indices = time_indices[int(len(time_indices) * train_ratio):int(len(time_indices) * (train_ratio + valid_ratio))]
test_indices = time_indices[int(len(time_indices) * (train_ratio + valid_ratio)):]

train_df = df.loc[train_indices, :]
valid_df = df.loc[valid_indices, :]
test_df = df.loc[test_indices, :]

print("Train start and end dates: ", train_indices[0], train_indices[-1])
try:
    print("Validation start and end dates: ", valid_indices[0], valid_indices[-1])
except:
    pass
print("Test start and end dates: ", test_indices[0], test_indices[-1])
scaler = MinMaxScaler()
train_df = pd.DataFrame(scaler.fit_transform(train_df), index=train_df.index, columns=train_df.columns)
valid_df = pd.DataFrame(scaler.transform(valid_df), index=valid_df.index, columns=valid_df.columns)
test_df = pd.DataFrame(scaler.transform(test_df), index=test_df.index, columns=test_df.columns)

969
Train start and end dates:  2019-01-26 03:00:00 2021-06-22 09:00:00
Validation start and end dates:  2021-06-22 10:00:00 2021-10-10 04:00:00
Test start and end dates:  2021-10-10 05:00:00 2022-01-27 23:00:00


In [13]:
def wmape(y_true, y_pred):
    total_abs_diff = tf.reduce_sum(tf.abs(tf.subtract(y_true, y_pred)))
    total = tf.reduce_sum(y_true)
    wmape = tf.realdiv(total_abs_diff, total)
    return wmape

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_wmape', patience=10, mode='min', 
        verbose=0, restore_best_weights=True)


MAX_EPOCHS = 100

In [29]:
OUT_STEPS = 24

columns = [col for col in train_df.columns if col != "rt_plant_id"]

window = WindowGenerator(
    train_df=train_df, valid_df=valid_df, test_df=test_df, columns=columns,
    input_width=120, label_width=24, shift=0, label_columns=["production"])

### Dense

In [ ]:
def objective(trial):
    clear_session()

    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(multi_window.input_width, multi_window.feature_number)))
    model.add(layers.Dense(
        units=trial.suggest_categorical("units", [OUT_STEPS, OUT_STEPS*2]),
        kernel_initializer=tf.initializers.zeros()))
    model.add(layers.Dropout(rate=trial.suggest_categorical("rate", [0.1, 0.2])))
    model.add(layers.Flatten())
    model.add(layers.Dense(
        OUT_STEPS, 
        activation=trial.suggest_categorical("activation", ["relu", "softmax", "sigmoid"]),))
    model.add(layers.Reshape([OUT_STEPS, 1]))

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    loss = trial.suggest_categorical("loss", ["mae", "mse"]),
    model.compile(
        loss=loss, metrics=[wmape],
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
    )
    model.fit(
        window.train,
        validation_data=window.valid,
        epochs=MAX_EPOCHS,
        verbose=2,
        validation_steps=2,
        callbacks=[early_stopping],
    )
    score = model.evaluate(window.valid, verbose=1)
    return score[1]

In [34]:
model = models.Sequential()

model.add(layers.InputLayer(input_shape=(multi_window.input_width, multi_window.feature_number)))
model.add(layers.Dense(
    units=OUT_STEPS,
    kernel_initializer=tf.initializers.zeros()))
model.add(layers.Dropout(rate=0.1))
model.add(layers.Flatten())
model.add(layers.Dense(
    OUT_STEPS, 
    activation="sigmoid"))
model.add(layers.Reshape([OUT_STEPS, 1]))

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_wmape', patience=10, mode='min', 
    verbose=0, restore_best_weights=True)
learning_rate = 0.00019257
loss = "mae"
model.compile(
    loss=loss,
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
    metrics=[wmape],
)

model.fit(
    window.train,
    validation_data=window.valid,
    epochs=MAX_EPOCHS,
    verbose=1,
    callbacks=[early_stopping],
)

# Evaluate the model accuracy on the validation set.
score = model.evaluate(window.valid, verbose=1)
score = model.evaluate(window.test, verbose=1)

Epoch 1/100
328/328 [==============================] - 1s 3ms/step - loss: 0.2141 - wmape: 0.6818 - val_loss: 0.1581 - val_wmape: 0.3549
Epoch 2/100
328/328 [==============================] - 1s 3ms/step - loss: 0.1408 - wmape: 0.4482 - val_loss: 0.1049 - val_wmape: 0.2360
Epoch 3/100
328/328 [==============================] - 1s 2ms/step - loss: 0.1160 - wmape: 0.3698 - val_loss: 0.0986 - val_wmape: 0.2224
Epoch 4/100
328/328 [==============================] - 1s 2ms/step - loss: 0.1084 - wmape: 0.3454 - val_loss: 0.0952 - val_wmape: 0.2141
Epoch 5/100
328/328 [==============================] - 1s 2ms/step - loss: 0.1043 - wmape: 0.3326 - val_loss: 0.0924 - val_wmape: 0.2086
Epoch 6/100
328/328 [==============================] - 1s 3ms/step - loss: 0.1014 - wmape: 0.3231 - val_loss: 0.0890 - val_wmape: 0.1998
Epoch 7/100
328/328 [==============================] - 1s 3ms/step - loss: 0.0990 - wmape: 0.3155 - val_loss: 0.0893 - val_wmape: 0.2008
Epoch 8/100
328/328 [====================

### CNN

In [40]:
def objective(trial):
    clear_session()
    
    conv_filters = trial.suggest_categorical("filters", [16, 32, 64])
    conv_width = trial.suggest_categorical("kernel_size", [3, 5, 7])
    conv_activation = trial.suggest_categorical("activation", ["relu", "linear"])
    pooling = trial.suggest_categorical("pool_size", [2, 3])
    dense_activation = trial.suggest_categorical("dense_activation", ["relu", "softmax", "sigmoid"])
    
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(multi_window.input_width, multi_window.feature_number)))
    model.add(layers.Conv1D(
        filters=conv_filters, 
        kernel_size=conv_width, 
        activation=conv_activation, 
        padding="same"))
    model.add(layers.MaxPooling1D(pooling))
    model.add(layers.Conv1D(conv_filters, conv_width, activation=conv_activation))
    model.add(layers.MaxPooling1D(pooling))
    model.add(layers.Dense(OUT_STEPS, activation=dense_activation, kernel_initializer=tf.initializers.zeros()))
    model.add(layers.Flatten())
    model.add(layers.Dense(OUT_STEPS, activation=dense_activation))
    model.add(layers.Reshape([OUT_STEPS, 1]))

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    loss = trial.suggest_categorical("loss", ["mae", "mse"])

    model.compile(
        loss=loss, metrics=[wmape],
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
    )
    model.fit(
        window.train,
        validation_data=window.valid,
        epochs=MAX_EPOCHS,
        verbose=2,
        validation_steps=2,
        callbacks=[early_stopping],
    )
    score = model.evaluate(window.test, verbose=1)
    return score[1]

In [41]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-04-06 19:58:12,863] A new study created in memory with name: no-name-0a8800e4-40b9-4240-a1f4-31fbfec5fa0f


Epoch 1/100
328/328 - 2s - loss: 0.1284 - wmape: 0.9890 - val_loss: 0.1100 - val_wmape: 0.6178 - 2s/epoch - 6ms/step
Epoch 2/100
328/328 - 1s - loss: 0.0927 - wmape: 0.8392 - val_loss: 0.1115 - val_wmape: 0.5991 - 1s/epoch - 4ms/step
Epoch 3/100
328/328 - 1s - loss: 0.0859 - wmape: 0.8044 - val_loss: 0.1309 - val_wmape: 0.6345 - 1s/epoch - 4ms/step
Epoch 4/100
328/328 - 1s - loss: 0.0853 - wmape: 0.8015 - val_loss: 0.1291 - val_wmape: 0.6534 - 1s/epoch - 4ms/step
Epoch 5/100
328/328 - 1s - loss: 0.0850 - wmape: 0.7987 - val_loss: 0.1273 - val_wmape: 0.6164 - 1s/epoch - 4ms/step
Epoch 6/100
328/328 - 1s - loss: 0.0844 - wmape: 0.7954 - val_loss: 0.1087 - val_wmape: 0.6176 - 1s/epoch - 4ms/step
Epoch 7/100
328/328 - 1s - loss: 0.0835 - wmape: 0.7910 - val_loss: 0.1095 - val_wmape: 0.6153 - 1s/epoch - 4ms/step
Epoch 8/100
328/328 - 1s - loss: 0.0821 - wmape: 0.7816 - val_loss: 0.1056 - val_wmape: 0.6089 - 1s/epoch - 4ms/step
Epoch 9/100
328/328 - 1s - loss: 0.0800 - wmape: 0.7699 - val_lo

[I 2022-04-06 20:00:05,116] Trial 0 finished with value: 0.342912495136261 and parameters: {'filters': 32, 'kernel_size': 7, 'activation': 'linear', 'pool_size': 3, 'dense_activation': 'sigmoid', 'learning_rate': 2.0770172513364982e-05, 'loss': 'mse'}. Best is trial 0 with value: 0.342912495136261.


Epoch 1/100
328/328 - 3s - loss: 0.3156 - wmape: 1.0000 - val_loss: 0.4489 - val_wmape: 1.0000 - 3s/epoch - 9ms/step
Epoch 2/100
328/328 - 2s - loss: 0.3156 - wmape: 1.0000 - val_loss: 0.4712 - val_wmape: 1.0000 - 2s/epoch - 7ms/step
Epoch 3/100
328/328 - 2s - loss: 0.3156 - wmape: 1.0000 - val_loss: 0.4323 - val_wmape: 1.0000 - 2s/epoch - 7ms/step
Epoch 4/100
328/328 - 2s - loss: 0.3156 - wmape: 1.0000 - val_loss: 0.4341 - val_wmape: 1.0000 - 2s/epoch - 7ms/step
Epoch 5/100
328/328 - 2s - loss: 0.3156 - wmape: 1.0000 - val_loss: 0.4344 - val_wmape: 1.0000 - 2s/epoch - 7ms/step
Epoch 6/100
328/328 - 2s - loss: 0.3156 - wmape: 1.0000 - val_loss: 0.4372 - val_wmape: 1.0000 - 2s/epoch - 7ms/step
Epoch 7/100
328/328 - 2s - loss: 0.3156 - wmape: 1.0000 - val_loss: 0.4372 - val_wmape: 1.0000 - 2s/epoch - 7ms/step
Epoch 8/100
328/328 - 2s - loss: 0.3156 - wmape: 1.0000 - val_loss: 0.4624 - val_wmape: 1.0000 - 2s/epoch - 7ms/step
Epoch 9/100
328/328 - 2s - loss: 0.3156 - wmape: 1.0000 - val_lo

[I 2022-04-06 20:00:31,451] Trial 1 finished with value: 1.0 and parameters: {'filters': 64, 'kernel_size': 5, 'activation': 'relu', 'pool_size': 2, 'dense_activation': 'relu', 'learning_rate': 0.09413175560616269, 'loss': 'mae'}. Best is trial 0 with value: 0.342912495136261.


...

Epoch 1/100
328/328 - 2s - loss: 0.0909 - wmape: 0.8276 - val_loss: 0.1123 - val_wmape: 0.6597 - 2s/epoch - 6ms/step
Epoch 2/100
328/328 - 1s - loss: 0.0751 - wmape: 0.7384 - val_loss: 0.0545 - val_wmape: 0.4617 - 1s/epoch - 4ms/step
Epoch 3/100
328/328 - 1s - loss: 0.0506 - wmape: 0.5786 - val_loss: 0.0356 - val_wmape: 0.3492 - 1s/epoch - 4ms/step
Epoch 4/100
328/328 - 1s - loss: 0.0332 - wmape: 0.4545 - val_loss: 0.0277 - val_wmape: 0.3128 - 1s/epoch - 5ms/step
Epoch 5/100
328/328 - 2s - loss: 0.0266 - wmape: 0.3928 - val_loss: 0.0230 - val_wmape: 0.2747 - 2s/epoch - 5ms/step
Epoch 6/100
328/328 - 1s - loss: 0.0246 - wmape: 0.3710 - val_loss: 0.0262 - val_wmape: 0.2782 - 1s/epoch - 4ms/step
Epoch 7/100
328/328 - 2s - loss: 0.0235 - wmape: 0.3591 - val_loss: 0.0242 - val_wmape: 0.2919 - 2s/epoch - 5ms/step
Epoch 8/100
328/328 - 2s - loss: 0.0227 - wmape: 0.3517 - val_loss: 0.0214 - val_wmape: 0.2608 - 2s/epoch - 5ms/step
Epoch 9/100
328/328 - 2s - loss: 0.0222 - wmape: 0.3461 - val_lo

[I 2022-04-06 20:07:55,739] Trial 14 finished with value: 0.32770904898643494 and parameters: {'filters': 32, 'kernel_size': 7, 'activation': 'linear', 'pool_size': 3, 'dense_activation': 'sigmoid', 'learning_rate': 0.00011896767283138643, 'loss': 'mse'}. Best is trial 11 with value: 0.3107956051826477.


Epoch 1/100
328/328 - 2s - loss: 0.0790 - wmape: 0.7590 - val_loss: 0.0542 - val_wmape: 0.4194 - 2s/epoch - 6ms/step
Epoch 2/100
328/328 - 1s - loss: 0.0364 - wmape: 0.4745 - val_loss: 0.0262 - val_wmape: 0.2954 - 1s/epoch - 4ms/step
Epoch 3/100
328/328 - 2s - loss: 0.0251 - wmape: 0.3755 - val_loss: 0.0248 - val_wmape: 0.2907 - 2s/epoch - 5ms/step
Epoch 4/100
328/328 - 2s - loss: 0.0230 - wmape: 0.3548 - val_loss: 0.0237 - val_wmape: 0.2783 - 2s/epoch - 5ms/step
Epoch 5/100
328/328 - 1s - loss: 0.0218 - wmape: 0.3440 - val_loss: 0.0219 - val_wmape: 0.2621 - 1s/epoch - 4ms/step
Epoch 6/100
328/328 - 1s - loss: 0.0210 - wmape: 0.3353 - val_loss: 0.0247 - val_wmape: 0.2541 - 1s/epoch - 4ms/step
Epoch 7/100
328/328 - 1s - loss: 0.0204 - wmape: 0.3299 - val_loss: 0.0196 - val_wmape: 0.2441 - 1s/epoch - 5ms/step
Epoch 8/100
328/328 - 1s - loss: 0.0199 - wmape: 0.3259 - val_loss: 0.0217 - val_wmape: 0.2427 - 1s/epoch - 4ms/step
Epoch 9/100
328/328 - 1s - loss: 0.0195 - wmape: 0.3221 - val_lo

[I 2022-04-06 20:08:28,269] Trial 15 finished with value: 0.3222605586051941 and parameters: {'filters': 32, 'kernel_size': 7, 'activation': 'linear', 'pool_size': 3, 'dense_activation': 'sigmoid', 'learning_rate': 0.0002736748837171292, 'loss': 'mse'}. Best is trial 11 with value: 0.3107956051826477.


Number of finished trials: 16
Best trial:
  Value: 0.3107956051826477
  Params: 
    filters: 32
    kernel_size: 7
    activation: linear
    pool_size: 3
    dense_activation: sigmoid
    learning_rate: 0.0007056118072814242
    loss: mse


In [42]:
conv_filters = 32
conv_width = 7
conv_activation = "linear" # trial.suggest_categorical("activation", ["relu", "linear"])
pooling = 3 # trial.suggest_categorical("pool_size", [2, 3])
dense_activation = "sigmoid"  # trial.suggest_categorical("dense_activation", ["relu", "softmax", "sigmoid"])
learning_rate = 0.0007056
loss = "mae"

model = models.Sequential()
model.add(layers.InputLayer(input_shape=(multi_window.input_width, multi_window.feature_number)))
model.add(layers.Conv1D(
    filters=conv_filters, 
    kernel_size=conv_width, 
    activation=conv_activation, 
    padding="same"))
model.add(layers.MaxPooling1D(pooling))
model.add(layers.Conv1D(conv_filters, conv_width, activation=conv_activation))
model.add(layers.MaxPooling1D(pooling))
model.add(layers.Dense(OUT_STEPS, activation=dense_activation, kernel_initializer=tf.initializers.zeros()))
model.add(layers.Flatten())
model.add(layers.Dense(OUT_STEPS, activation=dense_activation))
model.add(layers.Reshape([OUT_STEPS, 1]))

model.compile(
    loss=loss, metrics=[wmape],
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
)
model.fit(
    window.train,
    validation_data=window.valid,
    epochs=MAX_EPOCHS,
    verbose=2,
    validation_steps=2,
    callbacks=[early_stopping],
)
score = model.evaluate(window.valid, verbose=1)
score = model.evaluate(window.test, verbose=1)

Epoch 1/100
328/328 - 2s - loss: 0.1726 - wmape: 0.5504 - val_loss: 0.1248 - val_wmape: 0.3009 - 2s/epoch - 5ms/step
Epoch 2/100
328/328 - 1s - loss: 0.1125 - wmape: 0.3583 - val_loss: 0.1140 - val_wmape: 0.2606 - 1s/epoch - 4ms/step
Epoch 3/100
328/328 - 1s - loss: 0.1052 - wmape: 0.3352 - val_loss: 0.1167 - val_wmape: 0.2424 - 1s/epoch - 4ms/step
Epoch 4/100
328/328 - 1s - loss: 0.1008 - wmape: 0.3216 - val_loss: 0.1514 - val_wmape: 0.3215 - 1s/epoch - 4ms/step
Epoch 5/100
328/328 - 1s - loss: 0.0968 - wmape: 0.3089 - val_loss: 0.1073 - val_wmape: 0.2617 - 1s/epoch - 4ms/step
Epoch 6/100
328/328 - 1s - loss: 0.0940 - wmape: 0.2997 - val_loss: 0.1120 - val_wmape: 0.2527 - 1s/epoch - 4ms/step
Epoch 7/100
328/328 - 1s - loss: 0.0918 - wmape: 0.2928 - val_loss: 0.1072 - val_wmape: 0.2388 - 1s/epoch - 4ms/step
Epoch 8/100
328/328 - 1s - loss: 0.0900 - wmape: 0.2871 - val_loss: 0.1101 - val_wmape: 0.2486 - 1s/epoch - 4ms/step
Epoch 9/100
328/328 - 1s - loss: 0.0883 - wmape: 0.2814 - val_lo